# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# !pip install pipelinehelper

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
# from pipelinehelper import PipelineHelper
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

random_state = 0

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Qesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Qesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Qesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disastermanagement.db')
df = pd.read_sql_table('labeledmessages', engine)

In [3]:
# split dataset into feature and target variables
X = df['message']
Y = df.drop(columns = ['id', 'message', 'original', 'genre'])

In [4]:
from sklearn.utils.multiclass import type_of_target
type_of_target(Y)

'multilabel-indicator'

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Method to process the text data into lemmatized tokens

    Args:
        text: text data to be processed

    Returns:
        list: clean_tokes list with tokens extracted from the processed text data 
    """
    # normalize case and remove leading/trailing white space and punctuation
    text = re.sub("\W"," ", text.lower().strip())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # initiate stopword
    stop_words = stopwords.words("english")
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token to
    # lemmatize and remove stopwords  
    clean_tokens = []
    
    for tok in tokens:
        if tok not in stop_words:

            clean_tok = lemmatizer.lemmatize(tok)
            clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Giving some models a shot

#### 3.1 Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def model_pipeline(model):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(model))
    ])
      
    return pipeline

#### 3.2. Train and evaluate the pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=random_state)

In [10]:
# instantiate the models to train
lr_pipeline = model_pipeline(LogisticRegression(random_state = random_state))
svc_pipeline = model_pipeline(LinearSVC(random_state = random_state))
nb_pipeline = model_pipeline(MultinomialNB())
knc_pipeline = model_pipeline(KNeighborsClassifier())
tree_pipeline = model_pipeline(DecisionTreeClassifier(random_state = random_state))
rf_pipeline = model_pipeline(RandomForestClassifier(random_state = random_state))

models_part1 = [lr_pipeline, svc_pipeline, nb_pipeline]
models_part2 = [knc_pipeline, tree_pipeline, rf_pipeline]

In [11]:
# train and validate models part 1
df_result1 = pd.DataFrame()

for model in models_part1:
    scores = cross_val_score(model, X_train, y_train,
                             scoring = 'f1_samples', cv = 5)
    f1score = scores
    mean = f1score.mean()
    std = f1score.std()
    df_result1 = df_result1.append({
                                  'MODEL': str(model.named_steps['clf'].estimator).split('(')[0], 'SCORES': f1score, 
                                  'MEAN_SCORES': mean, 'STD_SCORES': std 
                                  }, ignore_index=True)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [12]:
df_result1

,MEAN_SCORES,MODEL,SCORES,STD_SCORES
0,0.514625,LogisticRegression,"[0.513047708223, 0.50890900552, 0.517944604815...",0.003467
1,0.501321,LinearSVC,"[0.497336722115, 0.498439652359, 0.49720207704...",0.005166
2,0.470794,MultinomialNB,"[0.462157473731, 0.467792671585, 0.48257941258...",0.007669


In [13]:
# train and validate models part 2
df_result2 = pd.DataFrame()

for model in models_part2:
    scores = cross_val_score(model, X_train, y_train,
                             scoring = 'f1_samples', cv = 5, n_jobs=-1)
    f1score = scores
    mean = f1score.mean()
    std = f1score.std()
    df_result2 = df_result2.append({
                                  'MODEL': str(model.named_steps['clf'].estimator).split('(')[0], 'SCORES': f1score, 
                                  'MEAN_SCORES': mean, 'STD_SCORES': std 
                                  }, ignore_index=True)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [14]:
df_result2

,MEAN_SCORES,MODEL,SCORES,STD_SCORES
0,0.440248,KNeighborsClassifier,"[0.437373002227, 0.433996509004, 0.44772590159...",0.005180
1,0.461471,DecisionTreeClassifier,"[0.464829925804, 0.454389031791, 0.46416511469...",0.004158
2,0.476431,RandomForestClassifier,"[0.471439794621, 0.471389868978, 0.48552065950...",0.005240


In [15]:
df_result = pd.concat([df_result1, df_result2], ignore_index=True)
df_result

,MEAN_SCORES,MODEL,SCORES,STD_SCORES
0,0.514625,LogisticRegression,"[0.513047708223, 0.50890900552, 0.517944604815...",0.003467
1,0.501321,LinearSVC,"[0.497336722115, 0.498439652359, 0.49720207704...",0.005166
2,0.470794,MultinomialNB,"[0.462157473731, 0.467792671585, 0.48257941258...",0.007669
3,0.440248,KNeighborsClassifier,"[0.437373002227, 0.433996509004, 0.44772590159...",0.005180
4,0.461471,DecisionTreeClassifier,"[0.464829925804, 0.454389031791, 0.46416511469...",0.004158
5,0.476431,RandomForestClassifier,"[0.471439794621, 0.471389868978, 0.48552065950...",0.005240


In [16]:
# making a scores dataframe
# separating the scores column into 5
df_scores = df_result['SCORES'].apply(pd.Series)
df_scores.columns=['SCORE_fold1', 'SCORE_fold2', 'SCORE_fold3', 'SCORE_fold4', 'SCORE_fold5']
df_scores

,SCORE_fold1,SCORE_fold2,SCORE_fold3,SCORE_fold4,SCORE_fold5
0,0.513048,0.508909,0.517945,0.518345,0.514878
1,0.497337,0.498440,0.497202,0.510813,0.502811
2,0.462157,0.467793,0.482579,0.476700,0.464738
3,0.437373,0.433997,0.447726,0.444914,0.437228
4,0.464830,0.454389,0.464165,0.464917,0.459051
5,0.471440,0.471390,0.485521,0.478302,0.475501


In [17]:
# concatenate both dataframes
df_result = pd.concat([df_result, df_scores], axis=1)
df_result

,MEAN_SCORES,MODEL,SCORES,STD_SCORES,SCORE_fold1,SCORE_fold2,SCORE_fold3,SCORE_fold4,SCORE_fold5
0,0.514625,LogisticRegression,"[0.513047708223, 0.50890900552, 0.517944604815...",0.003467,0.513048,0.508909,0.517945,0.518345,0.514878
1,0.501321,LinearSVC,"[0.497336722115, 0.498439652359, 0.49720207704...",0.005166,0.497337,0.498440,0.497202,0.510813,0.502811
2,0.470794,MultinomialNB,"[0.462157473731, 0.467792671585, 0.48257941258...",0.007669,0.462157,0.467793,0.482579,0.476700,0.464738
3,0.440248,KNeighborsClassifier,"[0.437373002227, 0.433996509004, 0.44772590159...",0.005180,0.437373,0.433997,0.447726,0.444914,0.437228
4,0.461471,DecisionTreeClassifier,"[0.464829925804, 0.454389031791, 0.46416511469...",0.004158,0.464830,0.454389,0.464165,0.464917,0.459051
5,0.476431,RandomForestClassifier,"[0.471439794621, 0.471389868978, 0.48552065950...",0.005240,0.471440,0.471390,0.485521,0.478302,0.475501


In [18]:
# drop the original 'SCORES' column
df_result.drop('SCORES', axis=1, inplace=True)
df_result

,MEAN_SCORES,MODEL,STD_SCORES,SCORE_fold1,SCORE_fold2,SCORE_fold3,SCORE_fold4,SCORE_fold5
0,0.514625,LogisticRegression,0.003467,0.513048,0.508909,0.517945,0.518345,0.514878
1,0.501321,LinearSVC,0.005166,0.497337,0.498440,0.497202,0.510813,0.502811
2,0.470794,MultinomialNB,0.007669,0.462157,0.467793,0.482579,0.476700,0.464738
3,0.440248,KNeighborsClassifier,0.005180,0.437373,0.433997,0.447726,0.444914,0.437228
4,0.461471,DecisionTreeClassifier,0.004158,0.464830,0.454389,0.464165,0.464917,0.459051
5,0.476431,RandomForestClassifier,0.005240,0.471440,0.471390,0.485521,0.478302,0.475501


In [19]:
# setting the final dataset
df_result = df_result[['MODEL', 'SCORE_fold1', 'SCORE_fold2', 'SCORE_fold3',
                       'SCORE_fold4', 'SCORE_fold5','MEAN_SCORES', 'STD_SCORES']]

df_result

,MODEL,SCORE_fold1,SCORE_fold2,SCORE_fold3,SCORE_fold4,SCORE_fold5,MEAN_SCORES,STD_SCORES
0,LogisticRegression,0.513048,0.508909,0.517945,0.518345,0.514878,0.514625,0.003467
1,LinearSVC,0.497337,0.498440,0.497202,0.510813,0.502811,0.501321,0.005166
2,MultinomialNB,0.462157,0.467793,0.482579,0.476700,0.464738,0.470794,0.007669
3,KNeighborsClassifier,0.437373,0.433997,0.447726,0.444914,0.437228,0.440248,0.005180
4,DecisionTreeClassifier,0.464830,0.454389,0.464165,0.464917,0.459051,0.461471,0.004158
5,RandomForestClassifier,0.471440,0.471390,0.485521,0.478302,0.475501,0.476431,0.005240


### 4. Improve chosen models
Use grid search to find better parameters. 

#### 4.1 Build a machine learning pipeline

In [8]:
# models that we'll use in the final pipeline
lr_clf = LogisticRegression(random_state = random_state)
svc_clf = LinearSVC(random_state = random_state)
rf_clf = RandomForestClassifier(random_state = random_state)

In [13]:
# # code doesn't run due to workspace expiration

# # few model tunning with pipelinehelper

# def model_pipeline():
#     pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('classifier', PipelineHelper([
#             ('lr', MultiOutputClassifier(lr_clf)),
#             ('svc', MultiOutputClassifier(svc_clf)),
#             ('rf', MultiOutputClassifier(rf_clf))
#     ]))
# ])

#     return pipeline

# pipeline = model_pipeline()


# params = {
#     'vect__decode_error' : ['strict', 'ignore', 'replace'],
#     'tfidf__norm' : ['l1', 'l2'],
#     'classifier__selected_model': pipeline.named_steps['classifier'].generate({
#         'lr__estimator__penalty' : ['l1', 'l2'],
#         'lr__estimator__C' : [1, 10],
#         'svc__estimator__loss': ['hinge', 'squared_hinge'],
#         'svc__estimator__C' : [1, 10],
#         'svc__estimator__multi_class': ['ovr', 'crammer_singer'],
#         'rf__estimator__n_estimators': [10, 100],
#         'rf__estimator__min_samples_leaf' : [1, 3]       
#     })
# }

# cv = GridSearchCV(pipeline, param_grid = params, n_jobs=-1)
# cv.fit(X_train, y_train)
# cv.best_params_

In [9]:
def model_pipeline(model):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('classifier', MultiOutputClassifier(model))   
])

    return pipeline


pipeline1 = model_pipeline(lr_clf)
pipeline2 = model_pipeline(svc_clf)
pipeline3 = model_pipeline(rf_clf)

In [10]:
params1 = {
    'vect__decode_error' : ['strict', 'ignore', 'replace'],
    'tfidf__norm' : ['l1', 'l2'],
    'classifier__estimator__penalty' : ['l1', 'l2'],
    'classifier__estimator__C' : [0.1, 1, 5, 10],
    'classifier__estimator__max_iter' : [500, 1000, 5000]
}


params2 = {
    'vect__decode_error' : ['strict', 'ignore', 'replace'],
    'tfidf__norm' : ['l1', 'l2'],
    'classifier__estimator__loss' : ['hinge', 'squared_hinge'],
    'classifier__estimator__C' : [1, 10],
    'classifier__estimator__multi_class': ['ovr', 'crammer_singer']
}


params3 = {
    'vect__decode_error' : ['strict', 'ignore', 'replace'],
    'tfidf__norm' : ['l1', 'l2'],        
    'classifier__estimator__n_estimators': [10, 100],
    'classifier__estimator__min_samples_leaf' : [1, 3]
}


In [12]:
cv1 = GridSearchCV(pipeline1, params1, n_jobs=-1)
cv1.fit(X_train, y_train)

In [11]:
# cv2 = GridSearchCV(pipeline2, params2, n_jobs=-1)
# cv2.fit(X_train, y_train)

In [12]:
# cv3 = GridSearchCV(pipeline3, params3, n_jobs=-1)
# cv3.fit(X_train, y_train)

In [24]:
cv1.best_params_

{'classifier__estimator__C': 1,
 'classifier__estimator__penalty': 'l1',
 'tfidf__norm': 'l2',
 'vect__decode_error': 'strict'}

In [14]:
# cv2.best_params_

{'classifier__estimator__C': 1,
 'classifier__estimator__loss': 'hinge',
 'classifier__estimator__multi_class': 'ovr',
 'tfidf__norm': 'l2',
 'vect__decode_error': 'strict'}

In [15]:
# cv3.best_params_

{'classifier__estimator__min_samples_leaf': 1,
 'classifier__estimator__n_estimators': 100,
 'tfidf__norm': 'l2',
 'vect__decode_error': 'strict'}

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset.

In [22]:
y_pred = cv1.predict(X_test)

print(classification_report(y_test, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      3983
               request       0.82      0.55      0.66       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.79      0.65      0.72      2226
          medical_help       0.66      0.21      0.31       447
      medical_products       0.65      0.26      0.37       284
     search_and_rescue       0.71      0.15      0.24       170
              security       0.33      0.02      0.04       104
              military       0.64      0.23      0.34       189
                 water       0.81      0.58      0.67       371
                  food       0.82      0.69      0.75       597
               shelter       0.78      0.57      0.66       477
              clothing       0.76      0.40      0.53        77
                 money       0.57      0.21      0.30       112
        missing_people       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Export your model as a pickle file

In [18]:
pickle.dump(cv1, open('message_lr_classifier.pkl','wb'))

### 7. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.